### Project Title : Beer Recommendation

### Introduction
##### Many online businesses rely on customer reviews and ratings. It is especially important in the ecommerce industry where customer engagements are greatly impacted by ratings. Websites rely on rating data to power its recommendation engine to provide the best recommendations that are personalized and most relevant to the user and make profits. 
##### This project focuses on the beer industry/beer online shops. To recommend the users with their potential ideal product based on their past ratings or features of the beers will be the key motivation.

#### Import libraries

In [4]:
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window 
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import math
import os
os.environ["PYSPARK_PYTHON"] = "python3"
import urllib
from pyspark.sql import SparkSession
## Recommendation Engine 
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS as ml_als
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

#### Data ETL

In [6]:
## setup spark session
spark = SparkSession \
    .builder \
    .appName("beer review") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [7]:
## Create Dataframe and SQL Table
## load data into dataframe and create sql tables
beers = spark.read.load("/FileStore/tables/beer_reviews.csv", format='csv', header = True)
beers.createOrReplaceTempView("beer_reviews")

In [8]:
## Display raw data
display(beers.take(5))

brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
10325,Vecchio Birraio,1234817823,1.5,2,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5,47986
10325,Vecchio Birraio,1235915097,3,2.5,3,stcules,English Strong Ale,3,3,Red Moon,6.2,48213
10325,Vecchio Birraio,1235916604,3,2.5,3,stcules,Foreign / Export Stout,3,3,Black Horse Black Beer,6.5,48215
10325,Vecchio Birraio,1234725145,3,3,3.5,stcules,German Pilsener,2.5,3,Sausa Pils,5,47969
1075,Caldera Brewing Company,1293735206,4,4.5,4,johnmichaelsen,American Double / Imperial IPA,4,4.5,Cauldron DIPA,7.7,64883


In [9]:
## The dimension of the dataframe
print((beers.count(), len(beers.columns)))

(1586614, 13)

In [10]:
## Check Null
print("Is there missing value in the dataframe?")
print('beers: {}'.format(beers.count() == beers.na.drop().count()))

beers: False

#### In this project, 1.5 millions of beer review data from BeerAdvocate through Kaggle will be analyzed.
#### Reviewer information, beer Information and rating information are provided. 
##### Specifically, 
##### Beer information includes : Beer Name, Beer ABV, Beer ID, Beer Style, Brewery Name, Brewery ID.
##### Rating infomration includes : Overall Ratings, Aroma Ratings, Apprearence Ratings, Palate Ratings, Taste Ratings.
##### Reviewer information includes : Reviewer profilename, Review time.

In [12]:
## Show data types
beers.printSchema()

root
-- brewery_id: string (nullable = true)
-- brewery_name: string (nullable = true)
-- review_time: string (nullable = true)
-- review_overall: string (nullable = true)
-- review_aroma: string (nullable = true)
-- review_appearance: string (nullable = true)
-- review_profilename: string (nullable = true)
-- beer_style: string (nullable = true)
-- review_palate: string (nullable = true)
-- review_taste: string (nullable = true)
-- beer_name: string (nullable = true)
-- beer_abv: string (nullable = true)
-- beer_beerid: string (nullable = true)

In [13]:
## Convert from spark dataframe to pandas dataframe
pandasbeer = beers.toPandas()

In [14]:
## Change reviewer profilename to userid
listOfStr = pandasbeer['review_profilename'].tolist()
my_dict = { i: listOfStr[i] for i in range(0, len(listOfStr) )}
flipped_dict = dict(zip(my_dict.values(), my_dict.keys()))
pandasbeer['review_profilename'] = pandasbeer.review_profilename.map(flipped_dict)

In [15]:
## Convert back to spark dataframe
mybeer = spark.createDataFrame(pandasbeer)

### Exploratory data analysis
#### In the first part of this project, I am going to explore what kind of data we get. 
##### I focused on beer style, since it might be an interesting characteristic that affects people's choice of beer. Other attributes, for example ABV, Style, Brewery should also be thoroughly explored if time allowed.
##### In addition, since we also have Overall Ratings, Aroma Ratings, Apprearence Ratings, Palate Ratings, Taste Ratings information, it might be interesting to see the relationships between beer features and different categories of ratings. It would also be interesting to model the overall ratings using ratings from other categories.

In [17]:
## Size of distinct items
print ("number of distinct users", mybeer.select('review_profilename').distinct().count())
print ("number of distinct beers", mybeer.select('beer_name').distinct().count())
print ("number of distinct beer styles", mybeer.select('beer_style').distinct().count())
print ("number of distinct breweries", mybeer.select('brewery_name').distinct().count())
print ("number of distinct ABV", mybeer.select('beer_abv').distinct().count())

number of distinct users 33388
number of distinct beers 56857
number of distinct beer styles 104
number of distinct breweries 5743
number of distinct ABV 531

#### Explore beer styles

In [19]:
## Unique Beer Styles
unique_beer_style = mybeer.select('beer_style').distinct()
display(unique_beer_style)

In [20]:
## The number of type of beers in each beer style
df_style= mybeer \
                    .groupBy('beer_style').count() \
                    .orderBy('count', ascending = False)
display(df_style)

beer_style,count
American IPA,117586
American Double / Imperial IPA,85977
American Pale Ale (APA),63469
Russian Imperial Stout,54129
American Double / Imperial Stout,50705
American Porter,50477
American Amber / Red Ale,45751
Belgian Strong Dark Ale,37743
Fruit / Vegetable Beer,33861
American Strong Ale,31945


In [21]:
##Example of beers of belgian strong dark ale style
display(mybeer.where("beer_style like '%Belgian Strong Dark Ale%'"))

#### Explore the overall ratings of beers

In [23]:
## The number of ratings of each scores
df_rate= mybeer \
                    .groupBy('review_overall').count() \
                    .orderBy('review_overall', ascending = False)
display(df_rate)

review_overall,count
5,91320
4.5,324385
4,582764
3.5,301817
3,165644
2.5,58523
2,38225
1.5,12975
1,10954
0,7


#### The average rating of beers from each beer style

In [25]:
pandasbeer['review_overall'] = pandasbeer['review_overall'].apply(pd.to_numeric)
pandasbeer.groupby("beer_style")["review_overall"].mean().sort_values(ascending = False).head(10)

Out[12]: beer_style
American Wild Ale 4.093262
Gueuze 4.086287
Quadrupel (Quad) 4.071630
Lambic - Unblended 4.048923
American Double / Imperial Stout 4.029820
Russian Imperial Stout 4.023084
Weizenbock 4.007969
American Double / Imperial IPA 3.998017
Flanders Red Ale 3.992722
Rye Beer 3.981737
Name: review_overall, dtype: float64

#### The beers that have the most ratings

In [27]:
beerrated = pandasbeer.groupby('beer_name')['review_overall'].count().sort_values(ascending=False).head(5)

#### The number of ratings that each reviewer gave

In [29]:
pandasbeer.groupby('review_profilename')['review_overall'].count().sort_values(ascending=False).head(5) 

Out[15]: review_profilename
1586535 5817
1586244 4661
1586246 4617
1584562 3518
1586447 3497
Name: review_overall, dtype: int64

##### Although using this interesting dataset from BeerAdvocate, the project can go into many directions.
##### In the rest part of this project, I am going to dive directly into predicting beer ratings only from review ID, and ratings that they gave to other beers.

### Start to predict beer ratings using recommendation algorithms with Spark MLlib APIs

In [32]:
## Create a new dataframe only focusing on the beer, reviewer and the ratings that give.
df_rating_data = mybeer.select("review_profilename","beer_beerid","review_overall")
df_rating_data = df_rating_data \
            .withColumn("review_profilename", df_rating_data.review_profilename.cast(IntegerType())) \
            .withColumn("beer_beerid", df_rating_data.beer_beerid.cast(IntegerType())) \
            .withColumn("review_overall", df_rating_data.review_overall.cast(DoubleType())) 

In [33]:
## Train and test split
data, hold_out = df_rating_data.randomSplit([0.8, 0.2], seed = 7856)
data.cache()
hold_out.cache()

Out[22]: DataFrame[review_profilename: int, beer_beerid: int, review_overall: double]

#### Use ALS (Alternating Least Square) and collaborative filtering to predict the ratings for the movies
#### ALS machine learning model referred from 
##### https://spark.apache.org/docs/2.2.0/ml-collaborative-filtering.html

In [35]:
# Build the recommendation model using ALS on the training data

# Specify model with parameters
als = ml_als(userCol="review_profilename", itemCol="beer_beerid", ratingCol="review_overall", coldStartStrategy="drop")

# Use a ParamGridBuilder to construct a grid of parameters to search over.
paramGrid = ParamGridBuilder()\
    .addGrid(als.maxIter,[10]) \
    .addGrid(als.rank, [6, 8, 10, 12, 14])\
    .addGrid(als.regParam, [0.05, 0.1, 0,5, 1, 10])\
    .build()

# Evaluate the model by computing the RMSE on the test data
evaluator = RegressionEvaluator(metricName="rmse", labelCol="review_overall",
                                predictionCol="prediction")

tvs = CrossValidator(estimator=als,
                     estimatorParamMaps=paramGrid,
                     evaluator=evaluator,
                     numFolds = 5)

# fit data
myalsmodel = tvs.fit(data)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-3410363378860028> in <module> 
 21 
 22 # fit data 
 ---> 23 myalsmodel = tvs . fit ( data ) 

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 130 return self . copy ( params ) . _fit ( dataset ) 
 131 else : 
 --> 132 return self . _fit ( dataset ) 
 133 else : 
 134 raise ValueError("Params must be either a param map or a list/tuple of param maps, "

 /databricks/spark/python/pyspark/ml/tuning.py in _fit (self, dataset) 
 308 
 309 tasks = _parallelFitTasks ( est , train , eva , validation , epm , collectSubModelsParam ) 
 --> 310 for j , metric , subModel in pool . imap_unordered ( lambda f : f ( ) , tasks ) : 
 311 metrics [ j ] += ( metric / nFolds ) 
 312 metrics_all [ i ] [ j ] = metric

 /usr/lib/python3.7/multiprocessing/pool.py in next (self, timeout) 
 746 if success : 
 747 return value
 --> 748 raise value
 749 
 750 __next__ = next # XXX 

 /usr/lib/python3.7/multiprocessing/pool.py in worker (inqueue, outqueue, initializer, initargs, maxtasks, wrap_exception) 
 119 job , i , func , args , kwds = task
 120 try : 
 --> 121 result = ( True , func ( * args , ** kwds ) ) 
 122 except Exception as e : 
 123 if wrap_exception and func is not _helper_reraises_exception : 

 /databricks/spark/python/pyspark/ml/tuning.py in <lambda> (f) 
 308 
 309 tasks = _parallelFitTasks ( est , train , eva , validation , epm , collectSubModelsParam ) 
 --> 310 for j , metric , subModel in pool . imap_unordered ( lambda f : f ( ) , tasks ) : 
 311 metrics [ j ] += ( metric / nFolds ) 
 312 metrics_all [ i ] [ j ] = metric

 /databricks/spark/python/pyspark/ml/tuning.py in singleTask () 
 52 
 53 def singleTask ( ) : 
 ---> 54 index , model = next ( modelIter ) 
 55 metric = eva . evaluate ( model . transform ( validation , epm [ index ] ) ) 
 56 return index , metric , model if collectSubModel else None 

 /databricks/spark/python/pyspark/ml/base.py in __next__ (self) 
 60 raise StopIteration ( "No models remaining." ) 
 61 self . counter += 1 
 ---> 62 return index , self . fitSingleModel ( index ) 
 63 
 64 def next ( self ) : 

 /databricks/spark/python/pyspark/ml/base.py in fitSingleModel (index) 
 104 
 105 def fitSingleModel ( index ) : 
 --> 106 return estimator . fit ( dataset , paramMaps [ index ] ) 
 107 
 108 return _FitMultipleIterator ( fitSingleModel , len ( paramMaps ) ) 

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 128 elif isinstance ( params , dict ) : 
 129 if params : 
 --> 130 return self . copy ( params ) . _fit ( dataset ) 
 131 else : 
 132 return self . _fit ( dataset ) 

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit (self, dataset) 
 293 
 294 def _fit ( self , dataset ) : 
 --> 295 java_model = self . _fit_java ( dataset ) 
 296 model = self . _create_model ( java_model ) 
 297 return self . _copyValues ( model ) 

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit_java (self, dataset) 
 290 """
 291 self . _transfer_params_to_java ( ) 
 --> 292 return self . _java_obj . fit ( dataset . _jdf ) 
 293 
 294 def _fit ( self , dataset ) : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format

#### Evaluate the model by computing the RMSE on the test data

In [37]:
# Get the best model from cross validation, evaluate the best model on test data
best_model = myalsmodel.bestModel
predictions = best_model.transform(hold_out)
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

In [38]:
#Print evaluation metrics and model parameters
print ("RMSE = "+str(rmse))
print ("**Best Model**")
print (" Rank:"+str(best_model._java_obj.parent().getRank())), 
print (" MaxIter:"+str(best_model._java_obj.parent().getMaxIter())), 
print (" RegParam:"+str(best_model._java_obj.parent().getRegParam()))

#### Show predicted ratings

In [40]:
## Show predictions
predictions.show()

#### Generate the top 5 beer recommendations for each user
#### Generate the top 5 user recommendations for each beer

In [42]:
# Generate top 5 beer recommendations for user whose id is 666
userRecs = myalsmodel.bestModel.recommendForAllUsers(5).cache()
display(userRecs.filter(userRecs.userId == 666))

In [43]:
# Generate top 5 user recommendations for each beer
beerRecs = myalsmodel.bestModel.recommendForAllItems(5).cache()
display(beerRecs)

### Discussion
#### Matrix factorization with ALS algorithm was used in the rating prediction task.
#### I used five-fold cross validation to train the model, selected the best model and evaluated on the testing data. 
#### Parameters including max iteration, regularization parameter, and rank were selected using grid search method. 
#### The best model showing the rank number gives us an insight of the number of the latent features in the ALS algorithm. The feature might be related to beer style, abv, brewery, aroma, appearance, palate, tasting and etc. A thorough exploration could be done latter. The findings might help in reinforcing the model using other method to improve the model in the future.
#### The predicted beer ratings and recommendations for customers are also shown above, providing recommended beers to the specific customers.
#### More explorations could be done using this dataset in possible future projects.
##### I focused on beer style, since it might be an interesting characteristic that affects people's choice of beer. Other attributes, for example ABV, Style, Brewery should also be thoroughly explored if time allowed.
##### In addition, since we also have Overall Ratings, Aroma Ratings, Apprearence Ratings, Palate Ratings, Taste Ratings information, it might be interesting to see the relationships between beer features and different categories of ratings. It would also be interesting to model the overall ratings using ratings from other categories, at least plot the distribution of theses ratings or run a simple linear regression.

### Appendix for all the columns in the dataset 
##### Reviewer Information : Reviewer profilename, Review time
###### review_profilename 
###### review_time
##### Rating Information : Overall Ratings, Aroma Ratings, Apprearence Ratings, Palate Ratings, Taste Ratings
###### review_overall
###### review_aroma
###### review_appearance
###### review_palate
###### review_taste
##### Beer Information: Beer Name, Beer ABV, Beer ID, Beer Style, Brewery Name, Brewery ID
###### beer_name
###### beer_abv
###### beer_beerid
###### beer_style
###### brewery_name
###### brewery_id

### References 
##### https://www.kaggle.com/rdoume/beerreviews
##### https://hub.packtpub.com/building-recommendation-engine-spark/ 
##### https://www.analyticsvidhya.com/blog/2016/06/quick-guide-build-recommendation-engine-python/ 
##### https://blog.statsbot.co/recommendation-system-algorithms-ba67f39ac9a3